<h3>The skip-gram word2vec model</h3>

In [1]:
from keras.layers import Merge
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential
vocab_size = 5000
embed_size = 300

Using TensorFlow backend.


In [2]:
word_model = Sequential()
word_model.add(Embedding(vocab_size, embed_size,
embeddings_initializer="glorot_uniform",
input_length=1))
word_model.add(Reshape((embed_size, )))

In [3]:
context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
embeddings_initializer="glorot_uniform",
input_length=1))
context_model.add(Reshape((embed_size,)))

In [15]:
model = Sequential()
model.add(Merge([word_model, context_model], mode="dot"))
#model.add(Dense(1, init="glorot_uniform", activation="sigmoid")) # 수정함
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
model.compile(loss="mean_squared_error", optimizer="adam")

C:\Users\Matthew\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [5]:
from keras.preprocessing.text import *
from keras.preprocessing.sequence import skipgrams
text = "I love green eggs and ham ."

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [8]:
word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

In [11]:
wids = [word2id[w] for w in text_to_word_sequence(text)]
pairs, labels = skipgrams(wids, len(word2id))
print(len(pairs), len(labels))
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(id2word[pairs[i][0]], pairs[i][0],id2word[pairs[i][1]], pairs[i][1],labels[i]))

56 56
(green (4), and (2)) -> 0
(i (6), love (5)) -> 0
(love (5), and (2)) -> 0
(and (2), love (5)) -> 1
(green (4), eggs (3)) -> 1
(green (4), and (2)) -> 1
(eggs (3), green (4)) -> 0
(and (2), and (2)) -> 0
(eggs (3), and (2)) -> 1
(and (2), and (2)) -> 0


<h3>The CBOW word2vec model</h3>

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Lambda
from keras.layers.embeddings import Embedding
import keras.backend as K
vocab_size = 5000
embed_size = 300
window_size = 1

In [18]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embed_size,
embeddings_initializer='glorot_uniform',
input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=
(embed_size,)))
model.add(Dense(vocab_size, kernel_initializer='glorot_uniform',
activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam")

In [19]:
merge_layer = model.layers[0]
word_model = merge_layer.layers[0]
word_embed_layer = word_model.layers[0]
weights = word_embed_layer.get_weights()[0]

AttributeError: 'Embedding' object has no attribute 'layers'

In [20]:
weights = model.layers[0].get_weights()[0]